<a href="https://colab.research.google.com/github/mariamempor/GQS--CALVETTI/blob/main/lab03_testes_unit_rios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LAB 03 ATV SALA

In [ ]:
from google.colab import drive

In [ ]:
# célula 1: função a ser testada !

def soma(a, b):
  return a + b

In [ ]:
import unittest

class TestSoma(unittest.TestCase):
  def test_soma_positivo(self):
    self.assertEqual(soma(2, 2), 4)

  def test_soma_negativo(self):
    self.assertEqual(soma(-2, -2), -4)

  def test_soma_zero(self):
    self.assertEqual(soma(0, 0), 0)


if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)

# ATIVIDADES

1.  Escreva função is_par(n) que retorna True se n for par;
Testes: números par, ímpar, zero, número negativo.
2.  Função fatorial(n), retorna fatorial para n >= 0, levanta ValueError se n < 0;
Testes: fatorial(0) == 1, fatorial(5) == 120, fatorial(-1) levanta ValueError.
3.  Classe Conta com métodos depositar(amount) e sacar(amount).
Levanta ValueError em valores negativos e InsufficientFunds quando tentar
sacar mais do que tem;
Testes: depósito, saque com sucesso, saque insuficiente, entradas
inválidas.
4.  Função buscar_clima(cidade) que chama:
requests.get('https://api.exemplo/clima?cidade=...') e retorna temperatura;
Escreva testes de mock que chamem requests.get e validem que sua função trata
corretamente a resposta e exceções (ex.: quando json() não contém temperatura).

In [ ]:
import unittest
from unittest.mock import patch, Mock


# 1. Função is_par
# =========================
def is_par(n):
    return n % 2 == 0


# 2. Função fatorial
# =========================
def fatorial(n):
    if n < 0:
        raise ValueError("n deve ser >= 0")
    if n == 0:
        return 1
    resultado = 1
    for i in range(1, n+1):
        resultado *= i
    return resultado


# 3. Classe Conta
# =========================
class InsufficientFunds(Exception):
    pass

class Conta:
    def __init__(self, saldo=0):
        self.saldo = saldo

    def depositar(self, amount):
        if amount < 0:
            raise ValueError("Depósito deve ser positivo")
        self.saldo += amount

    def sacar(self, amount):
        if amount < 0:
            raise ValueError("Saque deve ser positivo")
        if amount > self.saldo:
            raise InsufficientFunds("Saldo insuficiente")
        self.saldo -= amount


# 4. Função buscar_clima
# =========================
import requests

def buscar_clima(cidade):
    try:
        resp = requests.get(f"https://api.exemplo/clima?cidade={cidade}")
        dados = resp.json()
        if "temperatura" not in dados:
            raise KeyError("Resposta não contém temperatura")
        return dados["temperatura"]
    except Exception as e:
        return f"Erro: {e}"


# Testes Unitários
# =========================
class TestFuncoes(unittest.TestCase):

    # Testes para is_par
    def test_is_par(self):
        self.assertTrue(is_par(2))
        self.assertFalse(is_par(3))
        self.assertTrue(is_par(0))
        self.assertTrue(is_par(-4))
        self.assertFalse(is_par(-5))

    # Testes para fatorial
    def test_fatorial(self):
        self.assertEqual(fatorial(0), 1)
        self.assertEqual(fatorial(5), 120)
        with self.assertRaises(ValueError):
            fatorial(-1)

    # Testes para Conta
    def test_conta(self):
        conta = Conta(100)

        conta.depositar(50)
        self.assertEqual(conta.saldo, 150)

        conta.sacar(30)
        self.assertEqual(conta.saldo, 120)

        with self.assertRaises(InsufficientFunds):
            conta.sacar(1000)

        with self.assertRaises(ValueError):
            conta.depositar(-10)

        with self.assertRaises(ValueError):
            conta.sacar(-5)

    # Testes para buscar_clima com mock
    @patch("requests.get")
    def test_buscar_clima_sucesso(self, mock_get):
        mock_resp = Mock()
        mock_resp.json.return_value = {"temperatura": 25}
        mock_get.return_value = mock_resp

        resultado = buscar_clima("São Paulo")
        self.assertEqual(resultado, 25)

    @patch("requests.get")
    def test_buscar_clima_sem_temperatura(self, mock_get):
        mock_resp = Mock()
        mock_resp.json.return_value = {"umidade": 80}
        mock_get.return_value = mock_resp

        resultado = buscar_clima("São Paulo")
        self.assertTrue("Erro:" in resultado)

    @patch("requests.get")
    def test_buscar_clima_erro_request(self, mock_get):
        mock_get.side_effect = Exception("Falha na conexão")
        resultado = buscar_clima("Rio")
        self.assertTrue("Erro:" in resultado)


# Executar
# =========================
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


......
----------------------------------------------------------------------
Ran 6 tests in 0.009s

OK
